<div align="center">

# **Siniestros**

</div>

## Librerias

In [1]:
library(readxl)
library(lubridate)
library(dplyr)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
source("../src/data/ajuste_dinero_ipc.R")

## Data

In [3]:
siniestros <- read_excel("../data/input/Siniestros_Hist.xlsx")

In [4]:
colnames(siniestros)

[1] "FECHASIN"         "VLRPRIMAPAG"      "VLRPRISUSCR"      "VLRASEGU"        
 [5] "VLRASEGU_ORIG"    "VLRPAGADO."       "VLRSININCUR"      "VLRDEDUCIBLE"    
 [9] "VLRRECOBRO"       "VLRRSVACONSTIAMP" "VLRRSVAPAGAMP"    "FECPAGOAMP"      
[13] "ESTADO_FINAL"     "COBERTURA_FINAL"

In [5]:
names(siniestros) <- c(
  "Fecha",
  "Prima_efectivamente_pagada_hasta_fecha_siniestro",
  "Prima_suscrita_incluida_cobertura_responsabilidad_civil",
  "Asegurado_otras_coberturas_distintas_responsabilidad_civil",
  "Asegurado_original",
  "Efectivamente_pagado_por_siniestro",
  "Siniestro_incurrido",
  "Deducible",
  "Recobro_obtenido",
  "Reserva_constituida_para_amparo",
  "Reserva_pagada_para_amparo",
  "Fecha_pago_amparo",
  "Estado_final_siniestro",
  "Cobertura"
)

In [6]:
unique(siniestros$Cobertura)

[1] "PTH"       "PPD"       "RC BIENES" "PPH"       "RC PERS"

In [7]:
siniestros$Cobertura <- factor(siniestros$Cobertura,
                               levels = c("PTH", "PPD", "RC BIENES", "PPH", "RC PERS"),
                               labels = c("pth", 
                                          "ppd",
                                          "rc",
                                          "pph",
                                          "rc"))

Juntamos ambas responsabilidades civiles para que concuerde con las coberturas de polizas_v2.txt

## Fechas

In [8]:
siniestros$Fecha <- as.Date(siniestros$Fecha)

In [9]:
unique(format(siniestros$Fecha, "%Y"))

[1] "2017" "2018"

In [10]:
unique(format(siniestros$Fecha[year(siniestros$Fecha) == 2017], "%m"))

[1] "01"

In [11]:
unique(format(siniestros$Fecha[year(siniestros$Fecha) == 2018], "%m"))

[1] "01" "02" "03" "04" "05" "06" "07" "08" "09" "10" "11" "12"

Vamos a ignorar 2017 porque solo hay datos de enero

In [12]:
siniestros <- siniestros[year(siniestros$Fecha) == 2018, ]

## Nulos

In [13]:
sum(is.na(siniestros))

[1] 5380

In [14]:
colSums(is.na(siniestros))

Fecha 
                                                         0 
          Prima_efectivamente_pagada_hasta_fecha_siniestro 
                                                         0 
   Prima_suscrita_incluida_cobertura_responsabilidad_civil 
                                                         0 
Asegurado_otras_coberturas_distintas_responsabilidad_civil 
                                                         0 
                                        Asegurado_original 
                                                         0 
                        Efectivamente_pagado_por_siniestro 
                                                         0 
                                       Siniestro_incurrido 
                                                         0 
                                                 Deducible 
                                                         0 
                                          Recobro_obtenido 
                                                         0 
                           Reserva_constituida_para_amparo 
                                                         0 
                                Reserva_pagada_para_amparo 
                                                         0 
                                         Fecha_pago_amparo 
                                                      5380 
                                    Estado_final_siniestro 
                                                         0 
                                                 Cobertura 
                                                         0

Fecha_pago_amparo no es relevante para la modelacion que estamos llevando a cabo

## Eliminando duplicados

In [15]:
siniestros <- unique(siniestros)

## Seleccion de columnas

Vamos a seleccionar Siniestro_incurrido como la severidad, pues es el valor neto de los siniestros sin haber descontado el pago del deducible u otras posibles alteraciones del valor de cada accidente

In [16]:
siniestros <- siniestros[, c("Fecha", "Siniestro_incurrido", "Cobertura")]
names(siniestros) <- c("fecha", "severidad", "cobertura")
siniestros$semana <- week(siniestros$fecha)
siniestros$dia <- yday(siniestros$fecha)

## Llevando la severidad a enero de 2019

In [17]:
siniestros <- ajuste_dinero_ipc(siniestros, "fecha", "severidad")

## Segmentando por cobertura y eliminado valores extraños

- pph

In [18]:
siniestros_pph <- siniestros[(siniestros$cobertura) == "pph", ]
siniestros_pph$cobertura <- NULL
head(sort(unique(siniestros_pph$severidad)),20)

[1]      0.00  35245.61  41120.40  48747.69  64963.42  84273.51  89809.13
 [8] 115978.22 116139.55 142873.71 148001.55 158309.19 161411.76 162588.43
[15] 167265.75 171020.88 173384.72 178223.05 179107.02 194332.18

In [19]:
nrow(siniestros_pph[(siniestros_pph$severidad < 70000),])*100/nrow(siniestros_pph)

[1] 14.88498

In [20]:
siniestros_pph <- siniestros_pph[(siniestros_pph$severidad >= 70000),]

In [21]:
write.csv(siniestros_pph$severidad, "../data/processed/severidad_pph.csv")

- pth

In [22]:
siniestros_pth <- siniestros[(siniestros$cobertura) == "pth", ]
siniestros_pth$cobertura <- NULL
head(sort(unique(siniestros_pth$severidad)),20)

[1]      0.00  98013.54 116282.09 117180.52 160672.58 168071.23 201423.28
 [8] 279360.22 300007.33 312625.65 347420.97 347837.87 347906.76 348776.53
[15] 352879.00 353715.49 355943.90 378353.92 382935.47 385347.96

In [23]:
nrow(siniestros_pth[(siniestros_pth$severidad < 3000000),])*100/nrow(siniestros_pth)

[1] 21.60883

In [24]:
siniestros_pth <- siniestros_pth[(siniestros_pth$severidad >= 3000000),]

In [25]:
write.csv(siniestros_pth$severidad, "../data/processed/severidad_pth.csv")

- ppd

In [26]:
siniestros_ppd <- siniestros[(siniestros$cobertura) == "ppd", ]
siniestros_ppd$cobertura <- NULL
head(sort(unique(siniestros_ppd$severidad)) ,20)

[1]     0.000  1724.491  3430.260  3941.680  5907.670  9412.643 14352.093
 [8] 14636.996 16161.209 20491.290 23296.270 24189.512 27606.212 33332.915
[15] 37942.136 39237.076 43948.893 44300.778 45189.162 46270.689

In [27]:
nrow(siniestros_ppd[(siniestros_ppd$severidad < 70000),])*100/nrow(siniestros_ppd)

[1] 8.921866

In [28]:
siniestros_ppd <- siniestros_ppd[(siniestros_ppd$severidad >= 70000),]

In [29]:
write.csv(siniestros_ppd$severidad, "../data/processed/severidad_ppd.csv")

- rh

In [30]:
siniestros_rc <- siniestros[(siniestros$cobertura) == "rc", ]
siniestros_rc$cobertura <- NULL
head(sort(unique(siniestros_rc$severidad)),20)

[1]      0.00  19868.00  22486.97  30910.40  69359.54  73845.87  75315.27
 [8]  75526.30  75616.93  76352.57  83579.59  89083.77 100785.39 107481.00
[15] 117587.33 122164.11 122856.11 123784.91 123939.16 127099.54

In [31]:
nrow(siniestros_rc[(siniestros_rc$severidad < 500000),])*100/nrow(siniestros_rc)

[1] 18.6086

In [32]:
siniestros_rc <- siniestros_rc[(siniestros_rc$severidad >= 500000),]

In [33]:
write.csv(siniestros_rc$severidad, "../data/processed/severidad_rc.csv")

## Agrupando por semanas

- pph

In [34]:
siniestros_pph_semana <- siniestros_pph %>%
  group_by(semana) %>%
  summarise(
    severidad = sum(severidad),
    siniestros = n()
  )

write.csv(siniestros_pph_semana, "../data/processed/siniestros_pph_semana.csv")

In [35]:
siniestros_pph_dia <- siniestros_pph %>%
  group_by(dia) %>%
  summarise(
    severidad = sum(severidad),
    siniestros = n()
  )

write.csv(siniestros_pph_dia, "../data/processed/siniestros_pph_dia.csv")

- pth

In [36]:
siniestros_pth_semana <- siniestros_pth %>%
  group_by(semana) %>%
  summarise(
    severidad = sum(severidad),
    siniestros = n()
  )

write.csv(siniestros_pth_semana, "../data/processed/siniestros_pth_semana.csv")

In [37]:
siniestros_pth_dia <- siniestros_pth %>%
  group_by(dia) %>%
  summarise(
    severidad = sum(severidad),
    siniestros = n()
  )

write.csv(siniestros_pth_dia, "../data/processed/siniestros_pth_dia.csv")

- ppd

In [38]:
siniestros_ppd_semana <- siniestros_ppd %>%
  group_by(semana) %>%
  summarise(
    severidad = sum(severidad),
    siniestros = n()
  )

write.csv(siniestros_ppd_semana, "../data/processed/siniestros_ppd_semana.csv")

In [39]:
siniestros_ppd_dia <- siniestros_ppd %>%
  group_by(dia) %>%
  summarise(
    severidad = sum(severidad),
    siniestros = n()
  )

write.csv(siniestros_ppd_dia, "../data/processed/siniestros_ppd_dia.csv")

- rc

In [40]:
siniestros_rc_semana <- siniestros_rc %>%
  group_by(semana) %>%
  summarise(
    severidad = sum(severidad),
    siniestros = n()
  )

write.csv(siniestros_rc_semana, "../data/processed/siniestros_rc_semana.csv")

In [41]:
siniestros_rc_dia <- siniestros_rc %>%
  group_by(dia) %>%
  summarise(
    severidad = sum(severidad),
    siniestros = n()
  )

write.csv(siniestros_rc_dia, "../data/processed/siniestros_rc_dia.csv")